In [28]:
import mpilot
import arcpy
import inspect
import pandas as pd
import geopandas as gpd
import os
import shutil
from shapely import wkt
from pyproj import CRS

src_file_path = inspect.getfile(lambda: None)
THIS_FOLDER = os.path.dirname(os.path.abspath(src_file_path))

In [29]:
df = pd.read_csv("out_data/eems/A_RU_12.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18176 entries, 0 to 18175
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Join_ID     18176 non-null  int64  
 1   apn         18175 non-null  object 
 2   Pr_Lg_Fr    18176 non-null  float64
 3   Pr_Fi_Ign   18176 non-null  float64
 4   Pc_Cg_LgFi  18176 non-null  float64
 5   Pc_Fi_Ign   18176 non-null  float64
 6   Fl_Fa       18176 non-null  float64
 7   Buil_Ar     18176 non-null  float64
 8   Slp         18176 non-null  float64
 9   Shape_Leng  18176 non-null  float64
 10  Shape_Area  18176 non-null  float64
 11  WUI_Interm  18176 non-null  int64  
 12  WUI_Interf  18176 non-null  int64  
 13  Mj_LU_Cat   18176 non-null  int64  
 14  Mj_Haz_Cat  18176 non-null  int64  
 15  Vg_Pct      18176 non-null  float64
 16  Buil_Rat    18176 non-null  float64
 17  geometry    18176 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 2.5+ MB


## EEMS Model A - Core

In [30]:
def set_cmd_a(reporting_units_path, output_path):
    base_str = f"""

# Read Commands
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        Description: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Mj_LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_LU_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        Description: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Mj_Haz_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_Haz_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        Description: Relative&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fi_Ign = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fi_Ign",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pc_Cg_LgFi = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pc_Cg_LgFi",
    DataType = "Float",
    Metadata = [
        DisplayName: Average&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability&nbsp;Raw,
        Description: Avg&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Lg&nbsp;Fire&nbsp;Prob&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Fl_Fa = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Fl_Fa",
    DataType = "Float",
    Metadata = [
        DisplayName: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        Description: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_LU_Cat, 
    RawValues = [0,1,2,3,4], 
    FuzzyValues = [1,0,0.1,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Not&nbsp;Land&nbsp;Type&nbsp;Buffer&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

Hz_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_Haz_Cat, 
    RawValues = [3,2,1,0,-2,-1], 
    FuzzyValues = [1,0.75,0.5,0,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Cvt = CvtFromFuzzy(
    InFieldName = LU_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Not&nbsp;Land&nbsp;Type&nbsp;Buffer,
        ColorMap: RdYlBu_r
    ]
)

Hz_Cat_Cvt = CvtFromFuzzy(
    InFieldName = Hz_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone,
        ColorMap: RdYlBu_r
    ]
)


# Normalizations

Nm_Fl_Fa = Normalize(
    InFieldName = Fl_Fa, 
    StartVal = 1, 
    EndVal = 0,
    Metadata = [
        DisplayName: Low&nbsp;Ecological&nbsp;Importance,
        ColorMap: RdYlBu_r
    ]
)


Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Pr_F_Ig = Normalize(
    InFieldName = Pr_Fi_Ign, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

Nm_PcC_LgF = Normalize(
    InFieldName = Pc_Cg_LgFi, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability,
        ColorMap: RdYlBu_r
    ]
)

# Operations

C_Fr_Rsk = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF], 
    Weights = [0.6, 0.2, 0.2],
    Metadata = [
        DisplayName: CBI&nbsp;Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

CNm_Fr_Rsk = Normalize(
    InFieldName = C_Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: CBI&nbsp;Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)

Fr_Rsk = WeightedSum(
    InFieldNames = [CNm_Fr_Rsk, Hz_Cat_Cvt], 
    Weights = [0.9, 0.1],
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fr_Rsk = Normalize(
    InFieldName = Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)


WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)


Hi_Pri = WeightedSum(
    InFieldNames = [Nm_Fr_Rsk, Nm_Fl_Fa, LU_Cat_Cvt, WUI_Rsk], 
    Weights = [0.3, 0.2, 0.25, 0.25],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Hi_Pri = Normalize(
    InFieldName = Hi_Pri, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [Join_ID, Nm_Hi_Pri, Hi_Pri, Nm_Fr_Rsk, Fr_Rsk, Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF, LU_Cat_Cvt, Nm_Fl_Fa, 
                   Fl_Fa, WUI_Rsk, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fi_Ign, Pc_Cg_LgFi, LU_Cat_Cvt, LU_Cat_Fz, Mj_LU_Cat,
                   Mj_Haz_Cat, Hz_Cat_Fz, Hz_Cat_Cvt, C_Fr_Rsk, CNm_Fr_Rsk]
)
    """
    return base_str

In [31]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_12.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output_v7A.csv")
base_str = set_cmd_a(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE_v7A.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

In [32]:
dir_path = os.path.join(THIS_FOLDER, "out_data/final/a")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output_v7A.csv"), pd.read_csv("out_data/eems/A_RU_12.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:2229")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/a/RPP_EEMS_v7a.shp")

In [33]:
f1 = open("EEMS_CMD_FILE_v7A.mpt", 'r')
f2 = open("EEMS_CMD_FILE_v7A2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [34]:
arcpy.ResetEnvironments()
gdb_name = "EEMS_7A.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/a/RPP_EEMS_v7a.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")
crs = arcpy.SpatialReference('NAD 1983 UTM Zone 11N')
FeatureClass = arcpy.Project_management("EEMS", "EEMS_rpj", crs)
arcpy.management.Delete("EEMS")
arcpy.ClearWorkspaceCache_management()
del FeatureClass

## EEMS Model B

In [35]:
def set_cmd_b(reporting_units_path, output_path):
    base_str = f"""

# Read Commands
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        Description: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Mj_LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_LU_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        Description: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Mj_Haz_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_Haz_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        Description: Relative&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fi_Ign = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fi_Ign",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pc_Cg_LgFi = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pc_Cg_LgFi",
    DataType = "Float",
    Metadata = [
        DisplayName: Average&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability&nbsp;Raw,
        Description: Avg&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Lg&nbsp;Fire&nbsp;Prob&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Fl_Fa = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Fl_Fa",
    DataType = "Float",
    Metadata = [
        DisplayName: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        Description: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Buil_Rat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Buil_Rat",
    DataType = "Float",
    Metadata = [
        DisplayName: Building&nbsp;Site&nbsp;Ratio,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_LU_Cat, 
    RawValues = [0,1,2,3,4], 
    FuzzyValues = [1,0,0.1,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Not&nbsp;Land&nbsp;Type&nbsp;Buffer&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

Hz_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_Haz_Cat, 
    RawValues = [3,2,1,0,-2,-1], 
    FuzzyValues = [1,0.75,0.5,0,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Cvt = CvtFromFuzzy(
    InFieldName = LU_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Not&nbsp;Land&nbsp;Type&nbsp;Buffer,
        ColorMap: RdYlBu_r
    ]
)

Hz_Cat_Cvt = CvtFromFuzzy(
    InFieldName = Hz_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone,
        ColorMap: RdYlBu_r
    ]
)


# Normalizations

Nm_Fl_Fa = Normalize(
    InFieldName = Fl_Fa, 
    StartVal = 1, 
    EndVal = 0,
    Metadata = [
        DisplayName: Low&nbsp;Ecological&nbsp;Importance,
        ColorMap: RdYlBu_r
    ]
)


Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Pr_F_Ig = Normalize(
    InFieldName = Pr_Fi_Ign, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

Nm_PcC_LgF = Normalize(
    InFieldName = Pc_Cg_LgFi, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability,
        ColorMap: RdYlBu_r
    ]
)

NmBuil_Rat = CvtToFuzzyMeanToMid(
    InFieldName = Buil_Rat, 
    IgnoreZeros = False, 
    FuzzyValues = [1, 0.75, 0.5, 0.25, 0],
    Metadata = [
        DisplayName: Undevelopment&nbsp;Index&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

NFzBul_Rat = CvtFromFuzzy(
    InFieldName = NmBuil_Rat, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Undevelopment&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)


# Operations

C_Fr_Rsk = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF], 
    Weights = [0.6, 0.2, 0.2],
    Metadata = [
        DisplayName: CBI&nbsp;Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

CNm_Fr_Rsk = Normalize(
    InFieldName = C_Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: CBI&nbsp;Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)

Fr_Rsk = WeightedSum(
    InFieldNames = [CNm_Fr_Rsk, Hz_Cat_Cvt], 
    Weights = [0.9, 0.1],
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fr_Rsk = Normalize(
    InFieldName = Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)


WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)


Hi_Pri = WeightedSum(
    InFieldNames = [Nm_Fr_Rsk, Nm_Fl_Fa, LU_Cat_Cvt, WUI_Rsk], 
    Weights = [0.3, 0.2, 0.25, 0.25],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Hi_Pri = Normalize(
    InFieldName = Hi_Pri, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site,
        ColorMap: RdYlBu_r
    ]
)

#########

op_sp_re = Multiply(
    InFieldNames = [Nm_Hi_Pri, NFzBul_Rat],
    Metadata = [
        DisplayName: Openspace&nbsp;Wildfire&nbsp;Risk&nbsp;Reduction&nbsp;Priority&nbsp;Areas&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

os_hi_pri = Normalize(
    InFieldName = op_sp_re, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Openspace&nbsp;Wildfire&nbsp;Risk&nbsp;Reduction&nbsp;Priority&nbsp;Areas,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [Join_ID, Nm_Hi_Pri, Hi_Pri, Nm_Fr_Rsk, Fr_Rsk, Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF, LU_Cat_Cvt, Nm_Fl_Fa, 
                   Fl_Fa, WUI_Rsk, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fi_Ign, Pc_Cg_LgFi, LU_Cat_Cvt, LU_Cat_Fz, Mj_LU_Cat, 
                   Buil_Rat, NmBuil_Rat, op_sp_re, os_hi_pri, NFzBul_Rat, Mj_Haz_Cat, Hz_Cat_Fz, Hz_Cat_Cvt, C_Fr_Rsk, CNm_Fr_Rsk]
)
    """
    return base_str

In [36]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_12.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output_v7B.csv")
base_str = set_cmd_b(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE_v7B.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

In [37]:
dir_path = os.path.join(THIS_FOLDER, "out_data/final/b")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output_v7B.csv"), pd.read_csv("out_data/eems/A_RU_12.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:2229")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/b/RPP_EEMS_v7b.shp")

In [38]:
f1 = open("EEMS_CMD_FILE_v7B.mpt", 'r')
f2 = open("EEMS_CMD_FILE_v7B2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [39]:
arcpy.ResetEnvironments()
gdb_name = "EEMS_v7B.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/b/RPP_EEMS_v7b.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")
crs = arcpy.SpatialReference('NAD 1983 UTM Zone 11N')
FeatureClass = arcpy.Project_management("EEMS", "EEMS_rpj", crs)
arcpy.management.Delete("EEMS")
arcpy.ClearWorkspaceCache_management()
del FeatureClass

## EEMS Model C - Herbivory

In [40]:
def set_cmd_c(reporting_units_path, output_path):
    base_str = f"""
    
Join_ID = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Join_ID",
    DataType = "Integer",
    Metadata = [
        DisplayName: Join&nbsp;ID,
        Description: Join&nbsp;ID,
        ColorMap: RdYlBu_r
    ]
)

Mj_LU_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_LU_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        Description: Majority&nbsp;LandUse&nbsp;Category&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Mj_Haz_Cat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Mj_Haz_Cat",
    DataType = "Integer",
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Lg_Fr = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Lg_Fr",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        Description: Relative&nbsp;Probability&nbsp;Of&nbsp;Large&nbsp;Fire&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pr_Fi_Ign = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pr_Fi_Ign",
    DataType = "Float",
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Pc_Cg_LgFi = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Pc_Cg_LgFi",
    DataType = "Float",
    Metadata = [
        DisplayName: Average&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability&nbsp;Raw,
        Description: Avg&nbsp;Percent&nbsp;Change&nbsp;In&nbsp;Lg&nbsp;Fire&nbsp;Prob&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Fl_Fa = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Fl_Fa",
    DataType = "Float",
    Metadata = [
        DisplayName: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        Description: Flora&nbsp;And&nbsp;Fauna&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Buil_Rat = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Buil_Rat",
    DataType = "Float",
    Metadata = [
        DisplayName: Building&nbsp;Site&nbsp;Ratio,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interm = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interm",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Intermix,
        ColorMap: RdYlBu_r
    ]
)

WUI_Interf = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "WUI_Interf",
    DataType = "Integer",
    Metadata = [
        DisplayName: WUI&nbsp;Interface,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_LU_Cat, 
    RawValues = [0,1,2,3,4], 
    FuzzyValues = [1,0,0.1,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Not&nbsp;Land&nbsp;Type&nbsp;Buffer&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

Hz_Cat_Fz = CvtToFuzzyCat(
    InFieldName = Mj_Haz_Cat, 
    RawValues = [3,2,1,0,-2,-1], 
    FuzzyValues = [1,0.75,0.5,0,0,0], 
    DefaultFuzzyValue = 0,
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

LU_Cat_Cvt = CvtFromFuzzy(
    InFieldName = LU_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Not&nbsp;Land&nbsp;Type&nbsp;Buffer,
        ColorMap: RdYlBu_r
    ]
)

Hz_Cat_Cvt = CvtFromFuzzy(
    InFieldName = Hz_Cat_Fz, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Fire&nbsp;Hazard&nbsp;Severity&nbsp;Zone,
        ColorMap: RdYlBu_r
    ]
)


# Normalizations

Nm_Fl_Fa = Normalize(
    InFieldName = Fl_Fa, 
    StartVal = 1, 
    EndVal = 0,
    Metadata = [
        DisplayName: Low&nbsp;Ecological&nbsp;Importance,
        ColorMap: RdYlBu_r
    ]
)


Nm_Lg_Fire = Normalize(
    InFieldName = Pr_Lg_Fr, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Large&nbsp;Fire,
        ColorMap: RdYlBu_r
    ]
)

Nm_Pr_F_Ig = Normalize(
    InFieldName = Pr_Fi_Ign, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Probability&nbsp;Of&nbsp;Fire&nbsp;Ignition,
        ColorMap: RdYlBu_r
    ]
)

Nm_PcC_LgF = Normalize(
    InFieldName = Pc_Cg_LgFi, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Percent&nbsp;Change&nbsp;In&nbsp;Large&nbsp;Fire&nbsp;Probability,
        ColorMap: RdYlBu_r
    ]
)

NmBuil_Rat = CvtToFuzzyMeanToMid(
    InFieldName = Buil_Rat, 
    IgnoreZeros = False, 
    FuzzyValues = [1, 0.75, 0.5, 0.25, 0],
    Metadata = [
        DisplayName: Undevelopment&nbsp;Index&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)

NFzBul_Rat = CvtFromFuzzy(
    InFieldName = NmBuil_Rat, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Undevelopment&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)


# Operations

C_Fr_Rsk = WeightedSum(
    InFieldNames = [Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF], 
    Weights = [0.4, 0.4, 0.2],
    Metadata = [
        DisplayName: CBI&nbsp;Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

CNm_Fr_Rsk = Normalize(
    InFieldName = C_Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: CBI&nbsp;Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)

Fr_Rsk = WeightedSum(
    InFieldNames = [CNm_Fr_Rsk, Hz_Cat_Cvt], 
    Weights = [0.9, 0.1],
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Fr_Rsk = Normalize(
    InFieldName = Fr_Rsk, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Fire&nbsp;Risk&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)


WUI_Rsk = Maximum(
    InFieldNames = [WUI_Interm, WUI_Interf],
    Metadata = [
        DisplayName: WUI&nbsp;Risk,
        ColorMap: RdYlBu_r
    ]
)


Hi_Pri = WeightedSum(
    InFieldNames = [Nm_Fr_Rsk, Nm_Fl_Fa, LU_Cat_Cvt, WUI_Rsk], 
    Weights = [0.3, 0.2, 0.25, 0.25],
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

Nm_Hi_Pri = Normalize(
    InFieldName = Hi_Pri, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: High&nbsp;Priority&nbsp;Site,
        ColorMap: RdYlBu_r
    ]
)

#########

op_sp_re = Multiply(
    InFieldNames = [Nm_Hi_Pri, NFzBul_Rat],
    Metadata = [
        DisplayName: Openspace&nbsp;Wildfire&nbsp;Risk&nbsp;Reduction&nbsp;Priority&nbsp;Areas&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

os_hi_pri = Normalize(
    InFieldName = op_sp_re, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Openspace&nbsp;Wildfire&nbsp;Risk&nbsp;Reduction&nbsp;Priority&nbsp;Areas,
        ColorMap: RdYlBu_r
    ]
)

######################


Vg_Pct = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Vg_Pct",
    DataType = "Float",
    Metadata = [
        DisplayName: Vegetation&nbsp;Percentage,
        ColorMap: RdYlBu_r
    ]
)


Slp = EEMSRead(
    InFileName = {reporting_units_path},
    InFieldName = "Slp",
    DataType = "Integer",
    Metadata = [
        DisplayName: Slope&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)


Nm_Slp = CvtToFuzzyCurve(
    InFieldName = Slp, 
    RawValues = [0, 6, 16], 
    FuzzyValues = [0, 1, 0],
    Metadata = [
        DisplayName: Steeper&nbsp;Slope&nbsp;Index&nbsp;Fuzzy,
        ColorMap: RdYlBu_r
    ]
)


NFz_Slp = CvtFromFuzzy(
    InFieldName = Nm_Slp, 
    TrueThreshold = 1, 
    FalseThreshold = -1,
    Metadata = [
        DisplayName: Steeper&nbsp;Slope&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)

gt_idx_rw = Multiply(
    InFieldNames = [Vg_Pct, NFz_Slp],
    Metadata = [
        DisplayName: Herbivory&nbsp;Index&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

gt_idx = Normalize(
    InFieldName = gt_idx_rw, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Herbivory&nbsp;Index,
        ColorMap: RdYlBu_r
    ]
)

gt_wfra = Multiply(
    InFieldNames = [os_hi_pri, gt_idx],
    Metadata = [
        DisplayName: Herbivory&nbsp;Wildfire&nbsp;Risk&nbsp;Reduction&nbsp;Priority&nbsp;Areas&nbsp;Raw,
        ColorMap: RdYlBu_r
    ]
)

nmgt_wfra = Normalize(
    InFieldName = gt_wfra, 
    StartVal = 0, 
    EndVal = 1,
    Metadata = [
        DisplayName: Herbivory&nbsp;Wildfire&nbsp;Risk&nbsp;Reduction&nbsp;Priority&nbsp;Areas,
        ColorMap: RdYlBu_r
    ]
)

# Output

Out = EEMSWrite(
  OutFileName = {output_path},
  OutFieldNames = [nmgt_wfra, gt_wfra, gt_idx, gt_idx_rw, NFz_Slp, Nm_Slp, Slp, Vg_Pct, 
                   Join_ID, Nm_Hi_Pri, Hi_Pri, Nm_Fr_Rsk, Fr_Rsk, Nm_Lg_Fire, Nm_Pr_F_Ig, Nm_PcC_LgF, LU_Cat_Cvt, Nm_Fl_Fa, 
                   Fl_Fa, WUI_Rsk, WUI_Interm, WUI_Interf, Pr_Lg_Fr, Pr_Fi_Ign, Pc_Cg_LgFi, LU_Cat_Cvt, LU_Cat_Fz, Mj_LU_Cat, 
                   Buil_Rat, NmBuil_Rat, op_sp_re, os_hi_pri, NFzBul_Rat, Mj_Haz_Cat, Hz_Cat_Fz, Hz_Cat_Cvt, C_Fr_Rsk, CNm_Fr_Rsk]
)
    """
    return base_str

In [41]:
reporting_units_path = os.path.join(THIS_FOLDER, "out_data/eems/A_RU_12.csv")
output_path = os.path.join(THIS_FOLDER, "out_data/eems/EEMS_Output_v7C.csv")
base_str = set_cmd_c(reporting_units_path, output_path)
f = open("EEMS_CMD_FILE_v7C.mpt", "w")
f.write(base_str)
f.close()

**Run EEMS Command `mpilot eems-csv [command_file]`**

In [42]:
dir_path = os.path.join(THIS_FOLDER, "out_data/final/c")
shutil.rmtree(dir_path)
os.mkdir(dir_path)

eems = pd.merge(pd.read_csv("out_data/eems/EEMS_Output_v7C.csv"), pd.read_csv("out_data/eems/A_RU_12.csv")[["Join_ID", "geometry"]], on='Join_ID', how='inner')
eems['geometry'] = eems['geometry'].apply(wkt.loads)
crs_proj = CRS("EPSG:2229")
gdf = gpd.GeoDataFrame(eems, crs=crs_proj, geometry = eems.geometry)
gdf.to_file(driver = 'ESRI Shapefile', filename= "out_data/final/c/RPP_EEMS_v7c.shp")

In [43]:
f1 = open("EEMS_CMD_FILE_v7C.mpt", 'r')
f2 = open("EEMS_CMD_FILE_v7C2.mpt", 'w')
for line in f1:
    f2.write(line.replace('"', ''))
f1.close()
f2.close()

In [44]:
arcpy.ResetEnvironments()
gdb_name = "EEMS_v7C.gdb"
arcpy.env.overwriteOutput = True
path = os.path.join(THIS_FOLDER, gdb_name)
arcpy.CreateFileGDB_management("./", gdb_name)
arcpy.env.workspace = path
arcpy.env.scratchWorkspace = path
path = os.path.join(THIS_FOLDER, "out_data/final/c/RPP_EEMS_v7c.shp")
FeatureClass = arcpy.CopyFeatures_management(path, "EEMS")
crs = arcpy.SpatialReference('NAD 1983 UTM Zone 11N')
FeatureClass = arcpy.Project_management("EEMS", "EEMS_rpj", crs)
arcpy.management.Delete("EEMS")
arcpy.ClearWorkspaceCache_management()
del FeatureClass